In [79]:
from csv import reader
from collections import defaultdict
from itertools import chain, combinations


In [80]:
class TreeNode:
    def __init__(self, itemName, frequency, parentTreeNode):
        self.itemName = itemName
        self.count = frequency
        self.parent = parentTreeNode
        self.children = {}
        self.next = None

    def incrementCount(self, frequency):
        self.count += frequency

    def printTree(self, ind=1):
        print('  ' * ind, self.itemName, ' ', self.count)
        for child in list(self.children.values()):
            child.printTree(ind+1)

In [81]:
def readFile(fname):
    itemSetList = []
    frequency = []
    
    iter=0
    flg=0
    with open(fname, 'r') as file:
        csv_reader = reader(file)
        for line in csv_reader:
            if flg==0:
              flg=1
              continue
            if iter==100:
              break
            line = list(filter(None, line))
            itemSetList.append(line)
            frequency.append(1)
            iter=iter+1

    return itemSetList, frequency

In [82]:
def buildTree(itemSetList, frequency, minSup):
    headerTable = defaultdict(int)
    # Counting frequency and create header table
    for idx, itemSet in enumerate(itemSetList):
        for item in itemSet:
            headerTable[item] += frequency[idx]

    # Deleting items below minSup
    headerTable = dict((item, sup)
                       for item, sup in headerTable.items() if sup >= minSup)
    if(len(headerTable) == 0):
        return None, None

    # HeaderTable column [Item: [frequency, headTreeNode]]
    for item in headerTable:
        headerTable[item] = [headerTable[item], None]

    # Init Null head TreeNode
    fpTree = TreeNode('Null', 1, None)
    # Update FP tree for each cleaned and sorted itemSet
    for idx, itemSet in enumerate(itemSetList):
        itemSet = [item for item in itemSet if item in headerTable]
        itemSet.sort(key=lambda item: headerTable[item][0], reverse=True)
        # Traverse from root to leaf, update tree with given item
        currentTreeNode = fpTree
        for item in itemSet:
            currentTreeNode = updateTree(
                item, currentTreeNode, headerTable, frequency[idx])

    return fpTree, headerTable


def updateHeaderTable(item, targetTreeNode, headerTable):
    if(headerTable[item][1] == None):
        headerTable[item][1] = targetTreeNode
    else:
        currentTreeNode = headerTable[item][1]
        # Traverse to the last TreeNode then link it to the target
        while currentTreeNode.next != None:
            currentTreeNode = currentTreeNode.next
        currentTreeNode.next = targetTreeNode


def updateTree(item, treeNode, headerTable, frequency):
    if item in treeNode.children:
        # If the item already exists, incrementCount the count
        treeNode.children[item].incrementCount(frequency)
    else:
        # Create a new branch
        newItemTreeNode = TreeNode(item, frequency, treeNode)
        treeNode.children[item] = newItemTreeNode
        # Link the new branch to header table
        updateHeaderTable(item, newItemTreeNode, headerTable)

    return treeNode.children[item]


def ascendFPtree(TreeNode, prefixPath):
    if TreeNode.parent != None:
        prefixPath.append(TreeNode.itemName)
        ascendFPtree(TreeNode.parent, prefixPath)


def findPrefixPath(basePat, headerTable):
    # First TreeNode in linked list
    treeNode = headerTable[basePat][1]
    condPats = []
    frequency = []
    while treeNode != None:
        prefixPath = []
        # From leaf TreeNode all the way to root
        ascendFPtree(treeNode, prefixPath)
        if len(prefixPath) > 1:
            # Storing the prefix path and it's corresponding count
            condPats.append(prefixPath[1:])
            frequency.append(treeNode.count)

        # Go to next TreeNode
        treeNode = treeNode.next
    return condPats, frequency


def mineTree(headerTable, minSup, preFix, freqItemList):
    # Sort the items with frequency and create a list
    sortedItemList = [item[0] for item in sorted(
        list(headerTable.items()), key=lambda p:p[1][0])]
    # Start with the lowest frequency
    for item in sortedItemList:
        # Pattern growth is achieved by the concatenation of suffix pattern with frequent patterns generated from conditional FP-tree
        newFreqSet = preFix.copy()
        newFreqSet.add(item)
        freqItemList.append(newFreqSet)
        # Find all prefix path, constrcut conditional pattern base
        conditionalPattBase, frequency = findPrefixPath(item, headerTable)
        # Construct conditonal FP Tree with conditional pattern base
        conditionalTree, newHeaderTable = buildTree(
            conditionalPattBase, frequency, minSup)
        if newHeaderTable != None:
            # Mining recursively on the tree
            mineTree(newHeaderTable, minSup,
                     newFreqSet, freqItemList)


def powerset(s):
    return chain.from_iterable(combinations(s, r) for r in range(1, len(s)))


def getSupport(testSet, itemSetList):
    count = 0
    for itemSet in itemSetList:
        if(set(testSet).issubset(itemSet)):
            count += 1
    return count


def associationRule(freqItemSet, itemSetList, minConf):
    rules = []
    for itemSet in freqItemSet:
        subsets = powerset(itemSet)
        itemSetSup = getSupport(itemSet, itemSetList)
        for s in subsets:
            confidence = float(itemSetSup / getSupport(s, itemSetList))
            if(confidence > minConf):
              
               
                x = "Rule: {} ==> {} , {}.".format(set(s), set(itemSet.difference(s)), confidence)
               
                rules.append(x)
               
    return rules


def getFrequencyFromList(itemSetList):
    frequency = [1 for i in range(len(itemSetList))]
    return frequency

In [83]:
def fpgrowth(itemSetList, minSupRatio, minConf):
    frequency = getFrequencyFromList(itemSetList)
    minSup = len(itemSetList) * minSupRatio
    fpTree, headerTable = buildTree(itemSetList, frequency, minSup)
    if(fpTree == None):
        print('No frequent item set')
    else:
        freqItems = []
        mineTree(headerTable, minSup, set(), freqItems)
        rules = associationRule(freqItems, itemSetList, minConf)
        return freqItems, rules

In [84]:
def fpgrowthFromFile(fname, minSupRatio, minConf):
    l_table = [[] for i in range(4)]
    itemSetList, frequency = readFile(fname)
    minSup = len(itemSetList) * minSupRatio
    fpTree, headerTable = buildTree(itemSetList, frequency, minSup)
    if(fpTree == None):
        print('No frequent item set')
    else:
        freqItems = []
        freqItemset = []
        mineTree(headerTable, minSup, set(), freqItems)
        
        for i in freqItems:
            l_table[len(i)-1].append(i)
            # print(i)
            if len(i) == 4:
                freqItemset.append(i)
        rules = associationRule(freqItemset, itemSetList, minConf)
        return freqItemset, rules,l_table

In [85]:
minSup = 0.03
minConf = 0.7
freqItemSet, rules,l_table = fpgrowthFromFile("Goods_service_dataset.csv", minSup, minConf)
print(" *********** Intermediate Results *****************")
for j in range(0,4):
  print("L{} table: ".format(j+1))
  print(l_table[j])

    
print("")
print("Frequent Itemsets of length 4:\n")
print(freqItemSet, "\n")
print("Association rules:\n")
for i in range(len(rules)):
  print(rules[i])


 *********** Intermediate Results *****************
L1 table: 
[{'37'}, {'60'}, {'79'}, {'147'}, {'161'}, {'170'}, {'179'}, {'186'}, {'237'}, {'242'}, {'258'}, {'310'}, {'340'}, {'89'}, {'105'}, {'110'}, {'152'}, {'225'}, {'65'}, {'36'}, {'32'}, {'41'}, {'38'}, {'48'}, {'39'}]
L2 table: 
[{'37', '38'}, {'39', '79'}, {'147', '48'}, {'170', '38'}, {'170', '48'}, {'170', '39'}, {'39', '237'}, {'39', '242'}, {'89', '39'}, {'89', '48'}, {'105', '38'}, {'105', '39'}, {'110', '41'}, {'110', '38'}, {'110', '39'}, {'152', '39'}, {'48', '65'}, {'39', '65'}, {'41', '36'}, {'48', '36'}, {'39', '36'}, {'36', '38'}, {'38', '32'}, {'41', '32'}, {'48', '32'}, {'39', '32'}, {'41', '38'}, {'41', '48'}, {'41', '39'}, {'48', '38'}, {'39', '38'}, {'48', '39'}]
L3 table: 
[{'170', '48', '38'}, {'170', '48', '39'}, {'170', '39', '38'}, {'89', '48', '39'}, {'105', '39', '38'}, {'110', '39', '38'}, {'48', '39', '65'}, {'41', '36', '38'}, {'48', '36', '38'}, {'39', '36', '38'}, {'38', '39', '32'}, {'48', '38', 